# Know features

In [8]:
FEATURE_ORDER = [
    "store_id",
    "is_weekend",
    "is_holiday",
    "max_temp_c",
    "rainfall_mm",
    "is_hot_day",
    "is_rainy_day",
    "base_price",
    "discount_pct",
    "is_promo",
    "final_price",
    "year",
    "month",
    "day",
    "day_of_year",
    "day_of_week_index",
    "discount_amount",
    "is_promo_or_holiday",
]


# Single Inference

In [9]:
import pandas as pd
import numpy as np


def df_to_csv_payload(df: pd.DataFrame) -> str:
    """Reorder columns to FEATURE_ORDER and convert to headerless CSV string."""
    missing = [c for c in FEATURE_ORDER if c not in df.columns]
    if missing:
        raise ValueError(f"Missing features in input DataFrame: {missing}")

    X = df[FEATURE_ORDER].astype(np.float32).to_numpy()
    lines = [",".join(map(str, row)) for row in X]
    return "\n".join(lines)

# Example: build a one-row DataFrame (fake data)
df_request = pd.DataFrame([{
    "store_id": 1,
    "is_weekend": 0,
    "is_holiday": 0,
    "max_temp_c": 30.5,
    "rainfall_mm": 2.0,
    "is_hot_day": 1,
    "is_rainy_day": 0,
    "base_price": 12.5,
    "discount_pct": 0.1,
    "is_promo": 1,
    "final_price": 11.25,
    "year": 2024,
    "month": 2,
    "day": 15,
    "day_of_year": 46,
    "day_of_week_index": 3,
    "discount_amount": 1.25,
    "is_promo_or_holiday": 1,
}])

# payload = df_to_csv_payload(df_request)
# print("Payload:\n", payload)

# res = predictor.predict(payload)
# print("Raw response:", res)


In [12]:
import boto3

region = "us-east-1"  # adjust if needed
runtime_sm = boto3.client("sagemaker-runtime", region_name=region)

# Build payload string using the same FEATURE_ORDER / df_to_csv_payload
payload = df_to_csv_payload(df_request)

response = runtime_sm.invoke_endpoint(
    EndpointName="retail-demand-xgb-serverless-20251206-135312",
    ContentType="text/csv",
    Body=payload.encode("utf-8"),
)

body = response["Body"].read().decode("utf-8")
print("Raw body:", body)


Raw body: 88.05634307861328



# Multishot Inference

In [13]:
import pandas as pd
import boto3
import sagemaker

region = "us-east-1"  # adjust if needed
runtime_sm = boto3.client("sagemaker-runtime", region_name=region)

sess = sagemaker.Session()
bucket = sess.default_bucket()

# Reload test set (processed)
%store -r processed_test_data_s3_uri
print("processed_test_data_s3_uri:", processed_test_data_s3_uri)

test_csv_s3_path = processed_test_data_s3_uri.rstrip("/") + "/test.csv"

# Download test.csv locally
!mkdir -p data_test
!aws s3 cp $test_csv_s3_path data_test/test.csv

df_test = pd.read_csv("data_test/test.csv")
df_sample = df_test.head(5)   # take first 5 rows


payload = df_to_csv_payload(df_sample)

# get a real-time SageMaker endpoint
response = runtime_sm.invoke_endpoint(
    EndpointName="retail-demand-xgb-serverless-20251206-135312",
    ContentType="text/csv",
    Body=payload.encode("utf-8"),
)

body = response["Body"].read().decode("utf-8")
print("Raw body:", body)


processed_test_data_s3_uri: s3://sagemaker-us-east-1-423623839320/sagemaker-scikit-learn-2025-12-03-07-46-41-610/output/retail-test
download: s3://sagemaker-us-east-1-423623839320/sagemaker-scikit-learn-2025-12-03-07-46-41-610/output/retail-test/test.csv to data_test/test.csv
Raw body: 80.28925323486328
92.33282470703125
124.36448669433594
83.60984802246094
77.35962677001953

